# Intermediate Lesson on Geospatial Data 

## Spatial Queries

<strong>Lesson Developers:</strong> Jayakrishnan Ajayakumar, Shana Crosson, Mohsen Ahmadkhani

#### Part 5 of 5

In [ ]:
# This code cell starts the necessary setup for Hour of CI lesson notebooks.
# First, it enables users to hide and unhide code by producing a 'Toggle raw code' button below.
# Second, it imports the hourofci package, which is necessary for lessons and interactive Jupyter Widgets.
# Third, it helps hide/control other aspects of Jupyter Notebooks to improve the user experience
# This is an initialization cell
# It is not displayed because the Slide Type is 'Skip'

from IPython.display import HTML, IFrame, Javascript, display
from ipywidgets import interactive
import ipywidgets as widgets
from ipywidgets import Layout

import getpass # This library allows us to get the username (User agent string)

# import package for hourofci project
import sys
sys.path.append('../../supplementary') # relative path (may change depending on the location of the lesson notebook)
# sys.path.append('supplementary')
import hourofci
try:
    import os
    os.chdir('supplementary')
except:
    pass

# load javascript to initialize/hide cells, get user agent string, and hide output indicator
# hide code by introducing a toggle button "Toggle raw code"
HTML(''' 
    <script type="text/javascript" src=\"../../supplementary/js/custom.js\"></script>
    
    <style>
        .output_prompt{opacity:0;}
    </style>
    
    <input id="toggle_code" type="button" value="Toggle raw code">
''')

### Containment Query
The function **st_contains(geometry A,geometry B)** returns true if geometry A completely contains geometry B


<img src = "supplementary/images/containment_detail.png" width = "600px">

In the figure below, the `st_contains(geometry_circle, geometry_star)` returns the stars that are contained by the circles.

<img src = "supplementary/images/PointInMultiplePolygons.png" width = "600px">

### Query: How many starbucks are there in Minnesota?
<br/>
<center><img src = "supplementary/images/137-44505.png" width = "200px"></center>

To perform such query, we need two spatial tables, a table that keeps the US states' information (`us_states` table), and another one holding Starbuck stores' inforamtion (`starbucks` table) as shown below:

In [ ]:

from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select statefp,name,geom as geometry from us_states limit 5',db)
table2 = pd.read_sql_query('select pk_uid,fid,geom as geometry from starbucks limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>US_STATES</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>STARBUCKS</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

To count all the STARBUCKS that are **within** the state of 'Minnesota' we can use the following query:

```sql
select count(*) as total_starbucks 
from us_states u,starbucks s 
where u.name = 'Minnesota' and st_contains(u.geom,s.geom)
```

From the last chapter you might recall that this is a join operation involving multiple tables. But unlike the examples we saw, there is no explicit key-based relationship between the two tables. 

So instead of using a key-based relationship for the join we are using the relationship between the geometries of the two tables for the join. Such type of joins are called **Spatial Joins**.

>**Spatial Join** - **Joins attributes from one table to another based on their spatial relationship**.

#### Let's dismantle the query!
The clause
```sql
where u.name = 'Minnesota' 
```
retrieves the row from the `us_states` table with a name 'Minnesota' and the  function
```sql
st_contains(u.geom,s.geom)
```

retrieves those rows from `starbucks` and `us_states` tables where the **geometry in `starbucks` table (which is `point` in this case) is contained by the geometry in `us_states` table (which is `polygon`)** (which in this case is Minnesota) and then
```sql
count(*) as total_starbucks
```
counts the number of rows returned as a result of the where clause and assign it a name total_starbucks


Run this query yourself:



In [ ]:
inp6 = Textarea(description='<b>Query:</b>', value="select count(*) as total_starbucks \nfrom us_states u,starbucks s \nwhere u.name = 'Minnesota' and st_contains(u.geom,s.geom)" , layout=Layout(width='40%', height='120px'))
button6 = Button(description="Execute!")          
Box6 = HBox([inp6, button6])

db = spatialite.connect('databases/spatialDB.sqlite')


def execute_query6(b):
    clear_output()
    button6.on_click(execute_query6)
    display(Box6)
    print('Please wait...')
    table16 = pd.read_sql_query(inp6.value,db)
    clear_output()
    button6.on_click(execute_query6)
    display(Box6)
    return display(table16)

button6.on_click(execute_query6)
display(Box6)



Now, let's modify the question as below:
### How many starbucks are there in *each* state?

The key difference here is that we are not selecting any particular state and we want our results to be **grouped by** the state names.

We can write this query as 

```sql
select u.name,count(*) as total_starbucks 
from us_states u,starbucks s 
where st_contains(u.geom,s.geom) 
group by u.name
```

If you compare this query to the previous one you can notice that the clause that checks the state name is removed. 


Run this query yourself below:

In [ ]:

inp7 = Textarea(description='<b>Query:</b>', value="select u.name,count(*) as total_starbucks \nfrom us_states u,starbucks s \nwhere st_contains(u.geom,s.geom) \ngroup by u.name", layout=Layout(width='40%', height='120px'))
button7 = Button(description="Execute!")
Box7 = HBox([inp7, button7])

def execute_query7(b):
    clear_output()
    button7.on_click(execute_query7)
    display(Box7)
    print('Please wait...')
    table17 = pd.read_sql_query(inp7.value,db)
    clear_output()
    button7.on_click(execute_query7)
    display(Box7)
    return display(table17)

button7.on_click(execute_query7)
display(Box7)



### Let's visualize the result!

In the previous slide you were able to return the number of Starbucks branches in each state. Now, let's see those branches on a map! In the map below you can change the state interactively, get the counts, and see them on the map.



In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,WidgetControl,Rectangle,basemap_to_tiles,basemaps,Polygon,GeoJSON,Choropleth
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
def stateChanged(slid):
    layer_group.clear_layers()
    stateGeomSql = f"SELECT ST_AsBinary(geom) as geom FROM us_states where name='{states.value}';"
    starbucksSql = f"""SELECT ST_AsBinary(s.geom) as geom FROM us_states u,starbucks s where u.name='{states.value}'
     and st_contains(u.geom,s.geom) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'starbucks' 
        AND search_frame = u.geom)"""
    gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4269').to_crs('EPSG:4326')
    starbucksgdf = gpd.GeoDataFrame.from_postgis(starbucksSql, db,crs = 'EPSG:4326')
    center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
    sMap.center = center
    sMap.zoom = 6
    geo_data = GeoJSON(data = json.loads(gdf.to_json()),style={'opacity': 1, 'dashArray': '9', 'fillOpacity': 0, 'weight': 1})
    layer_group.add_layer(geo_data)
    geo_data_starbucks = GeoJSON(data = json.loads(starbucksgdf.to_json()))
    layer_group.add_layer(geo_data_starbucks)
    counts.value = str(len(starbucksgdf))
    
sql = "SELECT name FROM us_states order by name;"
statedf = pd.read_sql_query(sql,db)
sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)
layer_group = LayerGroup()
sMap.add_layer(layer_group)
states = widgets.Dropdown(
    options=statedf.name.values,
    value=statedf.name.values[0],
    description='State:',
    disabled=False,
)
counts=widgets.Text(
    value='',
    placeholder='',
    description='Total:',
    disabled=True,
)
states.observe(stateChanged, 'value')
filterParams=HBox([sMap,VBox([states,counts])])
stateChanged(None)
filterParams

Now when we look at the results we might not see any spatial patterns as such. But this is where spatial data and spatial database shines.

To show the spatial distribution we use a particular type of map called **Choropleth map**. 

**Below is a choropleth map showing the spatial distribution of earthquakes**

In [ ]:
from ipyleaflet import Map,DrawControl,GeoJSON,LayerGroup
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
from branca.colormap import linear
import matplotlib.pyplot as plt
from ipyleaflet import Choropleth
disp = widgets.Output()
db = spatialite.connect('databases/spatialDB.sqlite')
stateDataSql = f"""SELECT u.stusps,count(*) as total_earthquakes from us_states u,earthquakes s
 where st_contains(u.geom,s.geometry) and s.rowid in(SELECT ROWID 
    FROM SpatialIndex
    WHERE f_table_name = 'earthquakes' 
        AND search_frame = u.geom) group by u.stusps"""
dat = pd.read_sql_query(stateDataSql,db)
stateSql = """SELECT stusps,st_asbinary(geom) as geom FROM us_states"""
#df=pd.read_sql_query(sql,db)
df = gpd.read_postgis(stateSql,db)
dat = df[['stusps']].merge(dat,on='stusps',how='left').fillna(0)
dat =  dict(zip(dat['stusps'].tolist(), dat['total_earthquakes'].tolist()))
jsondata = json.loads(df.to_json())
for feature in jsondata['features']:
    feature['id'] = feature['properties']['stusps']
layer = Choropleth(
    geo_data=jsondata,
    choro_data=dat,
    colormap=linear.OrRd_03,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})
sMap= Map(center=(41.482222, -81.669722), zoom=3,prefer_canvas =True)
sMap.add_layer(layer)
sMap

### Intersection Query

### Intersects
First we look at how to use intersect function to check whether two geometries intersect.

The function **st_intersects(geometry A,geometry B)** returns true if geometry A and geometry B intersect or touch at atleast a single point

<img src = "supplementary/images/intersect.png" width = "400px">



A real world example of `st_intersects()` would be to identify the houses that fall within a hazard zone. We would not only want the houses that are fall in the hazard zone but also those that has some portion of it inside.

<img src = "supplementary/images/hazard_zones.png" width = "400px">

Let's look at an interactive example. In the next slide, you can use either the line or the rectangle tool to draw your own geometry. The geometries (in this cases us_states) that intersects with the geometry you have drawn will be highlighted in green color. 

In [ ]:
from ipyleaflet import Map,DrawControl,GeoJSON,LayerGroup
import geojson
import json
from shapely.geometry import shape
import geopandas as gpd
import numpy as np
import time
import spatialite
matching=[]
def handle_draw(target, action, geo_json):
    global matching
    draw_group.clear_layers()
    geo_dat = GeoJSON(data = geo_json)
    draw_group.add_layer(geo_dat)
    dc.clear()
    g1 = geojson.loads(json.dumps(geo_json['geometry']))
    g2 = shape(g1)
    sql = f"SELECT stusps from us_states u where st_intersects(u.geom,ST_TRANSFORM(ST_GeomFromText('{g2.wkt}',4326),4269))"
    pdf = pd.read_sql_query(sql, db)
    matching = pdf.values
    for layer in stateGroup.layers:
        layer.style={'weight':2+np.random.random_sample()}
    
def styleChange(feature):
    props=feature['properties']
    if props['stusps'] in matching:
        return {'opacity': 1, 'fillOpacity': 0, 'weight': 1,'color':'green'}
    else:
        return {'opacity': 0.3, 'fillOpacity': 0, 'weight': 1,'color':'red'}

sMap= Map(center=(44.967243, -103.771556), zoom=8,prefer_canvas =True)
sMap.fit_bounds(((24.1, -126.1), (49.9, -64.4)))
dc = DrawControl(
    marker={},
    rectangle={"shapeOptions": {"color": "#0000FF",'fillOpacity':0}},
    circle={},
    circlemarker={},
    polygon={}
)
dc.on_draw(handle_draw)
sMap.add_control(dc)
draw_group = LayerGroup()
sMap.add_layer(draw_group)
stateGroup = LayerGroup()
sMap.add_layer(stateGroup)
db = spatialite.connect('databases/spatialDB.sqlite')
stateGeomSql = f"SELECT stusps,ST_AsBinary(geom) as geom FROM us_states;"
gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4269').to_crs('EPSG:4326')
sMap.zoom = 6
geo_data = GeoJSON(data = json.loads(gdf.to_json()),style={'opacity': 0.3, 'fillOpacity': 0, 'weight': 1,'color':'red'},style_callback=styleChange)
stateGroup.add_layer(geo_data)
sMap

### Intersection
While intersects check whether two geometries intersect, intersection returns the geometry shared by the two geometries.

The function **st_intersection(geometry A,geometry B)** returns the geometry shared by geometry A and geometry B

<img src = "supplementary/images/intersection.png" width = "500px">

Now we will look at a concrete example of using st_intersection

**Find total length of subway lines in each neighborhood**

<img src = "supplementary/images/intersection_example.png" width = "400px">

To make this query, we will need one more function named **st_length(geometry)** for calculating the length of a geometry. 

Following are the tables involved in this query:

In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select boroname,name,geom as geometry from nyc_neighborhoods limit 5',db)
table2 = pd.read_sql_query('select pk_uid,geometry from nyc_subway_lines limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_NEIGHBORHOODS</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_SUBWAY_LINES</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

And here is the query:

```sql
SELECT u.boroname,sum(ST_Length(st_intersection(u.geom,s.geometry))) as total_length 
from nyc_neighborhoods u,nyc_subway_lines s
where st_intersects(u.geom,s.geometry) 
group by u.boroname
```
Challenge: can you dismantle this query yourself?

Click *Execute!* to run this query!

In [ ]:
valz2 = """
SELECT u.boroname,sum(ST_Length(st_intersection(u.geom,s.geometry))) as total_length 
from nyc_neighborhoods u,nyc_subway_lines s
where st_intersects(u.geom,s.geometry) 
group by u.boroname
"""
inp72 = Textarea(description='<b>Query:</b>', value=valz2, layout=Layout(width='40%', height='120px'))
button72 = Button(description="Execute!")
Box72 = HBox([inp72, button72])

def execute_query72(b):
    clear_output()
    button72.on_click(execute_query72)
    display(Box72)
    print('Please wait...')
    table172 = pd.read_sql_query(inp72.value,db)
    clear_output()
    button72.on_click(execute_query72)
    display(Box72)
    return display(table172)

button72.on_click(execute_query72)
display(Box72)




### A Spatial Analysis Example!
Now that you are familiar with spatial functions like `st_buffer()` and `st_contain()` it's time to make an actual spatial analysis! 

#### Query: What are the number of homicides within 100 units radius of the NYC substations?

To make such query, we need the following tables:


In [ ]:
from ipywidgets import HBox, VBox,widgets,Layout,HTML
from IPython.display import display
db = spatialite.connect('databases/spatialDB.sqlite')
table1 = pd.read_sql_query('select pk_uid,name,geom as geometry from nyc_substations limit 5',db)
table2 = pd.read_sql_query('select pk_uid,weapon,year,geom as geometry from nyc_homicides limit 5',db)
table1_disp = widgets.Output()
table2_disp = widgets.Output()
table1_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_SUBSTATIONS</center></b>")
table2_header = widgets.HTML(value = f"<b><font color='red'><center>NYC_HOMICIDES</center></b>")
with table1_disp:
    display(table1)
with table2_disp:
    display(table2)
out=HBox([VBox([table1_header,table1_disp],layout = Layout(margin='0 100px 0 0')),VBox([table2_header,table2_disp])])
out

And the query is 

```sql
SELECT u.name,count(*) as total_homicides 
FROM nyc_substations u,
nyc_homicides s 
WHERE st_contains(st_buffer(u.geom,100),s.geom) 
GROUP BY u.name
```

Run this query below!

In [ ]:
valz = """
SELECT u.name,count(*) as total_homicides 
FROM nyc_substations u,
nyc_homicides s 
WHERE st_contains(st_buffer(u.geom,100),s.geom) 
GROUP BY u.name
"""
inp7 = Textarea(description='<b>Query:</b>', value=valz, layout=Layout(width='40%', height='120px'))
button7 = Button(description="Execute!")
Box7 = HBox([inp7, button7])

def execute_query7(b):
    clear_output()
    button7.on_click(execute_query7)
    display(Box7)
    print('Please wait...')
    table17 = pd.read_sql_query(inp7.value,db)
    clear_output()
    button7.on_click(execute_query7)
    display(Box7)
    return display(table17)

button7.on_click(execute_query7)
display(Box7)



### A Cool Example: An Interactive Hospital Finder Tool

**The following map allows you to know how many hospitals are there within a specific distance of where you select.**

Double click anywhere on the map to select a location and see the hospitals within the disstance set by the slider (default is 3000 meter). You can change the slider to change the buffer value.

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,Polygon,GeoJSON,Marker
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
coords=None

def handle_click(**kwargs):
    global coords
    if kwargs.get('type') == 'dblclick':
        layer_group.clear_layers()
        coords = kwargs.get('coordinates')
        layer_group.add_layer(Marker(location=coords))
        findHospitals()
        
def findHospitals():
    global coords
    if coords is not None:
        stateGeomSql = f"""SELECT st_asbinary(u.geom) as geom from hospitals u
 where st_contains(st_buffer(st_transform(MakePoint({coords[1]},{coords[0]},4326),3857),{radiusSlider.value}),
 st_transform(u.geom,3857))"""
        gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4326')
        if len(gdf)!=0:
            
            geo_data = GeoData(geo_dataframe = gdf,
                style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
                name = 'Release')
            layer_group.add_layer(geo_data)
            center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
            sMap.center = center
            sMap.zoom = 12

def radiusChanged(slider):
    findHospitals()
        
sMap= Map(center=(41.482222, -81.669722), zoom=15,prefer_canvas =True)

radiusSlider = widgets.IntSlider(
    value=3000,
    min=0,
    max=100000,
    step=500,
    description='Radius:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

radiusSlider.observe(radiusChanged, 'value')
layer_group = LayerGroup()
sMap.add_layer(layer_group)
sMap.on_interaction(handle_click)
filterParams=HBox([sMap,VBox([radiusSlider])])
filterParams

### Distance Function

The distance function is used to find the distance between two geometries

The function **st_distance(geometry A,geometry B)** returns the distance between geometry A and geometry B

<img src = "supplementary/images/distance_restaurant_example.png" width = "500px"> 

Let's look at an interactive tool in the next slide that uses this function under the hood!


In [ ]:
# What is the distance between Minneapolis to St. Paul? 
# valz2 = """SELECT st_distance(st_transform(MakePoint(-93.090,44.9537,4326),3857),st_transform(MakePoint(-93.265,44.977,4326), 3857)) as distance
#        """

### Another Cool Example: Show me the farthest US state from where I choose!

In the map below, double-click anywhere you want and see the farthest state from there. 

In [ ]:
from ipyleaflet import Map, DrawControl,GeoData,LayerGroup,Polygon,GeoJSON,Marker
from ipywidgets import Button, HBox, VBox,widgets,Layout,GridspecLayout,IntSlider,HTML
from IPython.display import display
import spatialite
import pandas as pd
import geopandas as gpd
import json
import time
db = spatialite.connect('databases/spatialDB.sqlite')
coords=None

def handle_click(**kwargs):
    global coords
    if kwargs.get('type') == 'dblclick':
        layer_group.clear_layers()
        coords = kwargs.get('coordinates')
        layer_group.add_layer(Marker(location=coords))
        findNearestGasStation()
        
def findNearestGasStation():
    global coords
    if coords is not None:
        stateGeomSql = f"""SELECT name,st_asbinary(u.geom) as geom,st_distance(u.geom,MakePoint({coords[1]},{coords[0]},4326)) as dist_to_loc 
        from us_states u 
        order by dist_to_loc desc 
        limit 1"""
        gdf = gpd.GeoDataFrame.from_postgis(stateGeomSql, db,crs = 'EPSG:4326')
        geo_data = GeoData(geo_dataframe = gdf,
            style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
            hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
            point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
            name = 'Release')
        layer_group.add_layer(geo_data)
        center = [gdf.centroid.y.values[0],gdf.centroid.x.values[0]]
        sMap.center = center
        sMap.zoom = 3
        
sMap= Map(center=(41.482222, -81.669722), zoom=4,prefer_canvas =True)
layer_group = LayerGroup()
sMap.add_layer(layer_group)
sMap.on_interaction(handle_click)
sMap


# Congratulations!


**You have finished an Hour of CI!**


But, before you go ... 

1. Please fill out a very brief questionnaire to provide feedback and help us improve the Hour of CI lessons. It is fast and your feedback is very important to let us know what you learned and how we can improve the lessons in the future.
2. If you would like a certificate, then please type your name below and click "Create Certificate" and you will be presented with a PDF certificate.

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="https://forms.gle/JUUBm76rLB8iYppN7">Take the questionnaire and provide feedback</a></font>



In [ ]:

# This code cell loads the Interact Textbox that will ask users for their name
# Once they click "Create Certificate" then it will add their name to the certificate template
# And present them a PDF certificate
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

from ipywidgets import interact

def make_cert(learner_name, lesson_name):
    cert_filename = 'hourofci_certificate.pdf'

    img = Image.open("../../../supplementary/hci-certificate-template.jpg")
    draw = ImageDraw.Draw(img)

    cert_font = ImageFont.load_default()
    
    cert_font   = ImageFont.truetype('../../../supplementary/cruft.ttf', 150)
    cert_fontsm = ImageFont.truetype('../../../supplementary/cruft.ttf', 80) 
    w,h = cert_font.getsize(learner_name)    
    draw.text( xy = (1650-w/2,1100-h/2), text = learner_name, fill=(0,0,0),font=cert_font)
    w,h = cert_fontsm.getsize(lesson_name)
    draw.text( xy = (1650-w/2,1100-h/2 + 750), text = lesson_name, fill=(0,0,0),font=cert_fontsm)
    img.save(cert_filename, "PDF", resolution=100.0)  
    
    return cert_filename


interact_cert=interact.options(manual=True, manual_name="Create Certificate")

@interact_cert(name="Your Name")
def f(name):
    print("Congratulations",name)
    filename = make_cert(name, 'Intermediate Geospatial Data')
    print("Download your certificate by clicking the link below.")
    

<font size="+1"><a style="background-color:blue;color:white;padding:12px;margin:10px;font-weight:bold;" href="supplementary/hourofci_certificate.pdf?download=1" download="supplementary/hourofci_certificate.pdf">Download your certificate</a></font>